<a href="https://colab.research.google.com/github/OsmanAydogan/AIToolsDemonsAppp/blob/main/Coffe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sys
import docplex.mp
from geopy.distance import great_circle

In [4]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x  # Boylam
        self.y = y  # Enlem
    def __str__(self):
        return f"P({self.x}_{self.y})"

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

In [5]:
# Deneme
a = XPoint(123123122,12312323)
print(str(a))

P(123123122_12312323)


In [6]:
# Deneme
b = NamedPoint("deneme",123123122,12312323)
print(str(b))

deneme


**geopy.distance modülü, iki coğrafi nokta arasındaki mesafeyi hesaplamak için kullanılır.**

In [7]:
try:
    import geopy.distance
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install geopy
    else:
        !pip install --user geopy

In [8]:
# Simple distance computation between 2 locations.
from geopy.distance import great_circle

def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles

In [9]:
# Deneme
nyc = XPoint(-74.0060, 40.7128)  # New York City
la = XPoint(-118.2437, 34.0522)  # Los Angeles

distance = get_distance(nyc, la)
print(distance)

2445.5627972925


In [10]:
def build_libraries_from_url(url, name_pos, lat_long_pos):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    libraries = []
    k = 1
    for location in myjson:
        uname = location[name_pos]
        try:
            latitude = float(location[lat_long_pos][1])
            longitude = float(location[lat_long_pos][2])
        except TypeError :
            latitude = longitude = None

        try:
            name = str(uname)
        except:
            name = "???"
            name = f"K_{name}_{k}"

        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            libraries.append(cp)
            k += 1
    return libraries

In [11]:
libraries = build_libraries_from_url('https://data.cityofchicago.org/api/views/x8fc-8rcq/rows.json?accessType=DOWNLOAD',name_pos=10,lat_long_pos=17)

In [12]:
print(f"There are {len(libraries)} public libraries in Chicago")


There are 81 public libraries in Chicago


In [13]:
for library in libraries:
    print(library)


11071 S. Hoyne Ave.
3401 W. Foster Ave.
8148 S. Stony Island Ave.
1350 W. 89th St.
5108 W. Belmont Ave.
1336 W. Taylor St.
5331 W. Devon Ave.
6151 S. Normal Blvd.
6000 N. Broadway St.
5724 W. North Ave.
642 W. 43rd St.
3400 S. Halsted St.
6 S. Hoyne Ave.
11001 S. Indiana Ave.
415 East 79th St.
1000 E. 73rd St.
5440 S. Racine Ave.
955 E. 131st St.
4020 W. 63rd St.
2111 W. 47th St.
4101 W. 79th St.
6348 S. Archer Ave.
3436 S. King Dr.
163 E. Pearson St.
733 N. Kedzie Ave.
1605 N. Troy St.
7506 S. Racine Ave.
4904 S. Lake Park Ave.
4300 W. North Ave.
1625 W. Chicago Ave.
5630 N. Lincoln Ave.
1150 W. Fullerton Ave.
1745 W. 63rd St.
5055 S. Archer Ave.
3048 E. 130th St.
4024 N. Elston Ave.
1805 S. Loomis St.
4455 N. Lincoln Ave.
2100 S. Wentworth Ave.
3353 W. 13th St.
3030 W. Fullerton Ave.
6907 N. Clark St.
6871 W. Belden Ave.
3104 N. Narragansett Ave.
1701 N. Milwaukee Ave.
6800 N. Western Ave.
644 W. Belmont Ave.
830 W. 119th St.
1962 W. 95th St.
115 S. Pulaski Rd.
2708 S. Pulaski Rd.
56

In [14]:
nb_shops = 5
print(f"Our library has {nb_shops} books.")


Our library has 5 books.


In [15]:
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        !pip install folium
    else:
        !pip install folium

In [16]:
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for library in libraries:
    lt = library.y
    lg = library.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm


After running the above code, the data is displayed but it is impossible to determine where to ideally open the coffee shops by just looking at the map.

Let's set up DOcplex to write and solve an optimization model that will help us determine where to locate the coffee shops in an optimal way.

In [17]:
import pulp

# Using PuLP to replace Docplex
# Define problem
mdl = pulp.LpProblem("coffee_shops", pulp.LpMinimize)

# Unique points
libraries = set(libraries)
coffeeshop_locations = libraries  # Candidate coffee shop locations are the same as library locations.

# Decision variables
coffeeshop_vars = pulp.LpVariable.dicts("is_coffeeshop", coffeeshop_locations, cat="Binary")
link_vars = {
    (c_loc, b): pulp.LpVariable(f"link_{c_loc}_{b}", cat="Binary")
    for c_loc in coffeeshop_locations
    for b in libraries
}
BIGNUM = 999999999
# Constraints
for c_loc in coffeeshop_locations:
    for b in libraries:
        if get_distance(c_loc, b) >= BIGNUM:
            mdl += link_vars[c_loc, b] == 0

for b in libraries:
    mdl += pulp.lpSum(link_vars[c_loc, b] for c_loc in coffeeshop_locations) == 1

for c_loc in coffeeshop_locations:
    for b in libraries:
        mdl += link_vars[c_loc, b] <= coffeeshop_vars[c_loc]

mdl += pulp.lpSum(coffeeshop_vars[c_loc] for c_loc in coffeeshop_locations) == nb_shops

# Objective: Minimize total distance
mdl += pulp.lpSum(
    link_vars[c_loc, b] * get_distance(c_loc, b)
    for c_loc in coffeeshop_locations
    for b in libraries
)

# Solve
assert mdl.solve() == pulp.LpStatusOptimal, "Model did not solve optimally"

# Results
total_distance = pulp.value(mdl.objective)
open_coffeeshops = [c_loc for c_loc in coffeeshop_locations if pulp.value(coffeeshop_vars[c_loc]) == 1]
edges = [
    (c_loc, b)
    for c_loc in coffeeshop_locations
    for b in libraries
    if pulp.value(link_vars[c_loc, b]) == 1
]

print("Total distance =", total_distance)
print("# coffee shops =", len(open_coffeeshops))
for c in open_coffeeshops:
    print("new coffee shop:", c)

# Visualization
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for coffeeshop in open_coffeeshops:
    folium.Marker(
        [coffeeshop.y, coffeeshop.x], icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(map_osm)

for b in libraries:
    if b not in open_coffeeshops:
        folium.Marker([b.y, b.x]).add_to(map_osm)

for (c, b) in edges:
    coordinates = [[c.y, c.x], [b.y, b.x]]
    map_osm.add_child(folium.PolyLine(coordinates, color="#FF0000", weight=5))

map_osm.save("maping.html")


Total distance = 207.15243384854097
# coffee shops = 5
new coffee shop: 6100 W. Irving Park Rd.
new coffee shop: 4455 N. Lincoln Ave.
new coffee shop: 2111 W. 47th St.
new coffee shop: 9525 S. Halsted St.
new coffee shop: 6 S. Hoyne Ave.
